In [1]:
# THIS IS THE PART OF CODE DESIGNATED CONFIG IN THE ORIGINAL, I HAVE IT IN the SAME NOTEBOOK

#https://www.pyimagesearch.com/2019/06/03/fine-tuning-with-keras-and-deep-learning/

# import the necessary packages
import os

# initialize the path to the *original* input directory of images
ORIG_INPUT_DATASET = "Food-11"
# initialize the base path to the *new* directory that will contain
# our images after computing the training and testing split
BASE_PATH = "dataset"

# define the names of the training, testing, and validation
# directories
TRAIN = "training"
TEST = "evaluation"
VAL = "validation"

# initialize the list of class label names
CLASSES = ["hit", "no_hit"]

# set the batch size when fine-tuning
BATCH_SIZE = 16
# initialize the label encoder file path and the output directory to
# where the extracted features (in CSV file format) will be stored
LE_PATH = os.path.sep.join(["output", "le.cpickle"])
BASE_CSV_PATH = "output"
# set the path to the serialized model after training
MODEL_PATH = os.path.sep.join(["output", "food11.model"])
# define the path to the output training history plots
UNFROZEN_PLOT_PATH = os.path.sep.join(["output", "unfrozen.png"])
WARMUP_PLOT_PATH = os.path.sep.join(["output", "warmup.png"])

NEXT 

Move the files to the correct File structure (done already)

In [2]:
# import the necessary packages
from imutils import paths
import shutil
import os
# loop over the data splits
for split in (TRAIN, TEST, VAL):
    # grab all image paths in the current split
    print("[INFO] processing '{} split'...".format(split))
    p = os.path.sep.join([ORIG_INPUT_DATASET, split])
    imagePaths = list(paths.list_images(p))
    #print(imagePaths)
    # loop over the image paths
    for imagePath in imagePaths:
        # extract class label from the filename
        filename = imagePath.split(os.path.sep)[-1]
        label = CLASSES[int(filename.split("_")[0])]
        #print(label)
        # construct the path to the output directory
        dirPath = os.path.sep.join([BASE_PATH, split, label])
        
        # if the output directory does not exist, create it
        if not os.path.exists(dirPath):
            os.makedirs(dirPath)
            
        # construct the path to the output image file and copy it
        p = os.path.sep.join([dirPath, filename])
        shutil.copy2(imagePath, p)

[INFO] processing 'training split'...
[INFO] processing 'evaluation split'...
[INFO] processing 'validation split'...



NEXT - TRAIN MODEL CODE

From here is the implementation of the training code.


In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.utils import class_weight
from sklearn.metrics import classification_report


from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os

In [3]:

def plot_training(H, N, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)

In [3]:
# derive the paths to the training, validation, and testing
# directories
trainPath = os.path.sep.join([BASE_PATH, TRAIN])
valPath = os.path.sep.join([BASE_PATH, VAL])
testPath = os.path.sep.join([BASE_PATH, TEST])

print(valPath)
# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))
print(totalTest)

dataset\validation
64


In [4]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [5]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
    trainPath,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE)

# initialize the validation generator
valGen = valAug.flow_from_directory(
    valPath,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)

# initialize the testing generator
testGen = valAug.flow_from_directory(
    testPath,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=BATCH_SIZE)

Found 192 images belonging to 2 classes.
Found 64 images belonging to 2 classes.
Found 64 images belonging to 2 classes.


Next


Determine class weights for training set

In [6]:
from collections import Counter


counter = Counter(trainGen.classes)                          
max_val = float(max(counter.values()))       
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
print(class_weights)

{0: 1.0, 1: 1.0}


In [8]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
# new fc activation for binary cross entropy
#headModel = Dense(1, activation="sigmoid")(headModel)
#old fc layer
headModel = Dense(len(CLASSES), activation="softmax")(headModel)

# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
    layer.trainable = False

In [10]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=5e-5, momentum=0.9)

from keras.optimizers import Adam
#opt = Adam(lr=0.001)

model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit(
    x=trainGen,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=valGen,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=200,
    class_weight=class_weights)   #added class weights code

# reset the testing generator and evaluate the network after
# fine-tuning just the network head
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict(x=testGen,
    steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
    target_names=testGen.class_indices.keys()))
#plot_training(H, 10, WARMUP_PLOT_PATH)

[INFO] compiling model...


Using TensorFlow backend.


[INFO] training head...
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
12/12 [==============================] - 28s 2s/step - loss: 4.9960 - acc: 0.4792 - val_loss: 7.5971 - val_acc: 0.4688
Epoch 2/200
12/12 [==============================] - 22s 2s/step - loss: 6.5212 - acc: 0.4792 - val_loss: 7.4866 - val_acc: 0.4688
Epoch 3/200
12/12 [==============================] - 22s 2s/step - loss: 6.0915 - acc: 0.5000 - val_loss: 7.5077 - val_acc: 0.5156
Epoch 4/200
12/12 [==============================] - 22s 2s/step - loss: 5.6937 - acc: 0.5052 - val_loss: 4.8149 - val_acc: 0.5625
Epoch 5/200
12/12 [==============================] - 24s 2s/step - loss: 6.7164 - acc: 0.5365 - val_loss: 8.0590 - val_acc: 0.5000
Epoch 6/200
12/12 [==============================] - 23s 2s/step - loss: 7.4285 - acc: 0.4688 - val_loss: 7.1036 - val_acc: 0.5000
Epoch 7/200
12/12 [==============================] - 23s 2s/step - loss: 5.2856 - acc: 0.5625 - val_loss: 6.5347 - val_acc: 0.5625
Epoch 8/200

In [12]:
model.save("cathID")

In [ ]:
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict(x=testGen,
    steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
    target_names=testGen.class_indices.keys()))
plot_training(H, 40, WARMUP_PLOT_PATH)

 
 NEXT
 
 unfreeze the final set of CONV layers in the base model layers:

In [15]:
# reset our data generators
trainGen.reset()
valGen.reset()
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True
# loop over the layers in the model and show which ones are trainable
# or not
for layer in baseModel.layers:
    print("{}: {}".format(layer, layer.trainable))
model.save("cathID_unfreeze")   

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x000001A0A4116EF0>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001A0A411A550>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001A0A411A780>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001A0A416FEB8>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001A0A411AC18>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001A0A41AAA20>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001A0A41CF668>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001A0A41CF048>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001A0A4205898>: False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001A0A423C860>: False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001A0A424F898>: False
<tensorf


NEXT

Retrain the final Conv block and FC layer

In [18]:
# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
from tensorflow import keras

reconstructed_model = keras.models.load_model("cathID_unfreeze")

print("[INFO] re-compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
reconstructed_model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy"])
# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
H = reconstructed_model.fit(
    x=trainGen,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=valGen,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=200)

[INFO] re-compiling model...
Epoch 1/200
12/12 [==============================] - 24s 2s/step - loss: 0.5101 - acc: 0.7604 - val_loss: 0.3471 - val_acc: 0.8438
Epoch 2/200
12/12 [==============================] - 21s 2s/step - loss: 0.4798 - acc: 0.7240 - val_loss: 0.3871 - val_acc: 0.8125
Epoch 3/200
12/12 [==============================] - 21s 2s/step - loss: 0.5293 - acc: 0.7031 - val_loss: 0.4143 - val_acc: 0.8594
Epoch 4/200
12/12 [==============================] - 21s 2s/step - loss: 0.4884 - acc: 0.7448 - val_loss: 0.3744 - val_acc: 0.8750
Epoch 5/200
12/12 [==============================] - 21s 2s/step - loss: 0.5336 - acc: 0.6875 - val_loss: 0.3789 - val_acc: 0.8750
Epoch 6/200
12/12 [==============================] - 21s 2s/step - loss: 0.4894 - acc: 0.7656 - val_loss: 0.4942 - val_acc: 0.7812
Epoch 7/200
12/12 [==============================] - 21s 2s/step - loss: 0.5537 - acc: 0.7031 - val_loss: 0.4260 - val_acc: 0.7969
Epoch 8/200
12/12 [==============================] - 2

In [19]:
model.save("cathID_final")

In [22]:
plot_training(H, 200, WARMUP_PLOT_PATH)

In [7]:
from tensorflow import keras
Cathmodel = keras.models.load_model('FT_CathModel')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
reconstructed_model.fit(
    x=trainGen,
    steps_per_epoch=totalTrain // BATCH_SIZE,
    validation_data=valGen,
    validation_steps=totalVal // BATCH_SIZE,
    epochs=200)



pred_train= model.predict(X_train)
scores = model.evaluate(X_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))

In [10]:
# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating after fine-tuning network...")
testGen.reset()
predIdxsIO = Cathmodel.predict(x=testGen,
	steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxsIO, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
#plot_training(H, 200, UNFROZEN_PLOT_PATH)
# serialize the model to disk
print("[INFO] serializing network...")
Cathmodel.save("FT_CathModel2")

[INFO] evaluating after fine-tuning network...
              precision    recall  f1-score   support

         hit       0.94      0.91      0.92        32
      no_hit       0.91      0.94      0.92        32

   micro avg       0.92      0.92      0.92        64
   macro avg       0.92      0.92      0.92        64
weighted avg       0.92      0.92      0.92        64

[INFO] serializing network...


In [15]:
print(predIdxsIO)

[[0.5599808  0.44001913]
 [0.5061579  0.49384212]
 [0.57738036 0.42261964]
 [0.8711092  0.12889086]
 [0.90454525 0.09545477]
 [0.70914876 0.2908512 ]
 [0.94729733 0.05270264]
 [0.98259026 0.01740968]
 [0.82804257 0.17195739]
 [0.85661745 0.14338258]
 [0.8536947  0.14630528]
 [0.85492605 0.14507397]
 [0.77278817 0.22721177]
 [0.90962124 0.09037879]
 [0.23429392 0.76570606]
 [0.632005   0.36799505]
 [0.9931745  0.00682552]
 [0.98850983 0.01149023]
 [0.53811246 0.4618875 ]
 [0.6086678  0.3913322 ]
 [0.60097885 0.39902112]
 [0.5510812  0.44891882]
 [0.9937272  0.00627277]
 [0.9327568  0.06724328]
 [0.7116435  0.2883565 ]
 [0.6925104  0.3074896 ]
 [0.46096995 0.5390301 ]
 [0.4812445  0.5187555 ]
 [0.5743035  0.4256965 ]
 [0.6564564  0.34354362]
 [0.75053567 0.24946432]
 [0.96859866 0.03140134]
 [0.25632414 0.7436758 ]
 [0.02888471 0.97111535]
 [0.34882975 0.65117025]
 [0.50930214 0.49069786]
 [0.16144253 0.8385574 ]
 [0.1541764  0.84582365]
 [0.13580097 0.86419904]
 [0.11133955 0.88866043]


In [16]:
print(testGen.classes)
pred_prob = predIdxsIO[:,1]
print(pred_prob)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0.44001913 0.49384212 0.42261964 0.12889086 0.09545477 0.2908512
 0.05270264 0.01740968 0.17195739 0.14338258 0.14630528 0.14507397
 0.22721177 0.09037879 0.76570606 0.36799505 0.00682552 0.01149023
 0.4618875  0.3913322  0.39902112 0.44891882 0.00627277 0.06724328
 0.2883565  0.3074896  0.5390301  0.5187555  0.4256965  0.34354362
 0.24946432 0.03140134 0.7436758  0.97111535 0.65117025 0.49069786
 0.8385574  0.84582365 0.86419904 0.88866043 0.88664794 0.9694297
 0.9631044  0.8490356  0.744536   0.74604046 0.888488   0.7825701
 0.6943307  0.95363605 0.93464315 0.92810804 0.5628603  0.8837424
 0.86643046 0.6015444  0.6286062  0.58715016 0.9182958  0.7576604
 0.7312427  0.79049736 0.7632103  0.1929038 ]


In [17]:
from sklearn.metrics import roc_curve
fpr_keras, tpr_keras, thresholds_keras = roc_curve(testGen.classes, pred_prob)

In [30]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)
print(auc_keras)

0.966796875


In [31]:
plt.figure(4)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.savefig("roc_fig_full2")

# Zoom in view of the upper left corner.
plt.figure(3)
plt.xlim(0, 0.2)
plt.ylim(0.8, 1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
#plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve (zoomed in at top left)')
plt.legend(loc='best')
plt.savefig("roc_fig2")